In [1]:
import os
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import optuna

import seaborn as sns



In [2]:
# 파일 호출
data_path: str = "data"
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train 
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)
df


,ID,target,_type
0,2023-01-01 00:00:00,2.0,train
1,2023-01-01 01:00:00,1.0,train
2,2023-01-01 02:00:00,1.0,train
3,2023-01-01 03:00:00,1.0,train
4,2023-01-01 04:00:00,2.0,train
...,...,...,...
2787,2024-04-26 03:00:00,NaN,test
2788,2024-04-26 04:00:00,NaN,test
2789,2024-04-26 05:00:00,NaN,test
2790,2024-04-26 06:00:00,NaN,test


In [3]:
file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
]
# 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
    _rename_rule = {
        col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")

100%|██████████| 107/107 [00:01<00:00, 53.67it/s]


In [4]:
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_gap": "coinbase_premium_gap",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_index": "coinbase_premium_index",
    "hourly_market-data_funding-rates_all_exchange_funding_rates": "funding_rates",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations": "long_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations_usd": "long_liquidations_usd",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations": "short_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations_usd": "short_liquidations_usd",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_ratio": "buy_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_sell_ratio": "buy_sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume": "buy_volume",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_ratio": "sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume": "sell_volume",
    "hourly_network-data_transactions-count_transactions_count_total" : 'transaction_count',
    'hourly_network-data_hashrate_hashrate' : "hashrate",
}
# HOURLY_MARKET-DATA_PRICE-OHLCV_ALL_EXCHANGE_SPOT_BTC_USD

df = df[cols_dict.keys()].rename(cols_dict, axis=1)
df.shape


(11552, 18)

In [5]:
df = df.assign(
    liquidation_diff=df["long_liquidations"] - df["short_liquidations"],
    liquidation_usd_diff=df["long_liquidations_usd"] - df["short_liquidations_usd"],
    volume_diff=df["buy_volume"] - df["sell_volume"],
    liquidation_diffg=np.sign(df["long_liquidations"] - df["short_liquidations"]),
    liquidation_usd_diffg=np.sign(df["long_liquidations_usd"] - df["short_liquidations_usd"]),
    volume_diffg=np.sign(df["buy_volume"] - df["sell_volume"]),
    buy_sell_volume_ratio=df["buy_volume"] / (df["sell_volume"] + 1),
    
)
df['volume_diff_pct_change_12h'] = df['volume_diff'].pct_change(periods=12)
df['volume_diff_pct_change_6h'] = df['volume_diff'].pct_change(periods=6)
df['volume_diff_pct_change_additive']=df['volume_diff_pct_change_6h']+df['volume_diff_pct_change_12h'] # star
df['mean_7days'] = df['volume_diff'].rolling(window=168).mean()  # 7 days if hourly data
df['mean_1day'] = df['volume_diff'].rolling(window=24).mean()    # 1 day if hourly data
df['slope_7days'] = df['mean_7days'].diff()
df['slope_1day'] = df['mean_1day'].diff()
df['slope_change']=df['slope_7days']-df['slope_1day'] # star
df.drop(columns=['mean_7days', 'mean_1day'], inplace=True)




/var/folders/zk/056hcq1553j8j8d2gphz8ckm0000gn/T/ipykernel_43849/1946267302.py:11: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['volume_diff_pct_change_12h'] = df['volume_diff'].pct_change(periods=12)
/var/folders/zk/056hcq1553j8j8d2gphz8ckm0000gn/T/ipykernel_43849/1946267302.py:12: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['volume_diff_pct_change_6h'] = df['volume_diff'].pct_change(periods=6)


In [6]:
df['liquidation_diff_pct_change_3h'] = (df['liquidation_diff']-df['liquidation_diff'].shift(3))/(df['liquidation_diff'].shift(3)+1)
df['liquidation_diff_pct_change_1h'] = (df['liquidation_diff']-df['liquidation_diff'].shift(1))/(df['liquidation_diff'].shift(1)+1)
df.groupby('target')['liquidation_diff_pct_change_1h'].mean()+df.groupby('target')['liquidation_diff_pct_change_3h'].mean()

target
0.0   -8.140666
1.0    0.101549
2.0   -7.368863
3.0   -2.307253
dtype: float64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11552 entries, 0 to 11551
Data columns (total 33 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               11552 non-null  object 
 1   target                           8760 non-null   float64
 2   _type                            11552 non-null  object 
 3   coinbase_premium_gap             11546 non-null  float64
 4   coinbase_premium_index           11546 non-null  float64
 5   funding_rates                    11520 non-null  float64
 6   long_liquidations                11552 non-null  float64
 7   long_liquidations_usd            11552 non-null  float64
 8   short_liquidations               11552 non-null  float64
 9   short_liquidations_usd           11552 non-null  float64
 10  open_interest                    11548 non-null  float64
 11  buy_ratio                        11521 non-null  float64
 12  buy_sell_ratio    

In [ ]:


def Null_fill(
    df: pd.DataFrame,
    method: str = 'mean'  # Default to 'mean'
) -> pd.DataFrame:
    """
    Fills missing values in the DataFrame based on the specified method.

    Parameters:
    - df: The DataFrame to fill missing values in.
    - method: The method to use for filling missing values. Options are 'mean', 'median', 'mode'.

    Returns:
    - df: The DataFrame with filled missing values.
    """
    # Loop through all columns in the DataFrame
    df_train=df[df['_type']=='train']
    for col in df.columns:
        # Check if the column contains any NaN values
        if df[col].isna().sum() != 0:
            # Choose the appropriate method for filling missing values
            if method == 'mean':
                val = df_train[col].mean()
            elif method == 'median':
                val = df_train[col].median()
            elif method == 'mode':
                val = df_train[col].mode()[0]  # Use the first mode if multiple
            else:
                raise ValueError(f"Invalid method: {method}. Choose 'mean', 'median', or 'mode'.")
            
            # Fill NaN values in the column
            df[col] = df[col].fillna(val)
    
    return df

In [11]:
df['ID']=pd.to_datetime(df.ID)
def change(x):
  if x < 8:
    return 1
  elif x< 12:
    return 2
  elif x < 18:
    return 3
  return 4
date_col = ["Weekday", "Weekend", "Hour","Hour_1","Weekend_Hour"]
df["Weekday"] = df.ID.dt.weekday.astype("str")
df["Weekend"] = df["Weekday"].apply( lambda x: "1" if x in ["5","6"] else "0")
df["Hour"] = df.ID.dt.hour.astype("str")
df["Hour_1"] = df.ID.dt.hour.map(change).astype("str")
df["Weekend_Hour"] = df["Weekend"] + df["Hour_1"]


In [12]:
df['Weekday'] = df['Weekday'].astype('int64')
df["Weekend"] = df["Weekend"].astype('int64')
df["Hour"] = df['Hour'].astype('int64')
df["Hour_1"] = df['Hour'].astype('int64')
df["Weekend_Hour"] = df['Weekend_Hour'].astype('int64')

In [13]:
category_cols: List[str] = ["liquidation_diffg", "liquidation_usd_diffg", "volume_diffg",'cross_volume_diff','cross_shifted_volume_diff',
                            'cross_signal_volume_diff','mean_diff_7days_1day_volume','mean_diff_1month_7days_volume',
                            'mean_diff_7days_1day_liquidation','mean_diff_1month_7days_liquidation',
                            'cross_liquidation_diff','cross_shifted_liquidation_diff',
                           'cross_signal_liquidation_diff','slope_7days','slope_1day','slope_1month',"Weekday","Weekend","Hour","Hour_1",
                           "Weekend_Hour",'sell_volume_pct_change','buy_volume_pct_change',"price_pct_change","price_change",'volume_diff_pct_change_12h',
 'volume_diff_pct_change_6h','sender_count',"active_count","receiver_count",'coinbase_premium_gap','liquidation_diff_pct_change_3h',
 'liquidation_diff_pct_change_1h',
 'coinbase_premium_index']
conti_cols: List[str] = [_ for _ in df.columns if _ not in ["ID", "target", "_type"]+category_cols]  





In [14]:
conti_cols

['funding_rates',
 'long_liquidations',
 'long_liquidations_usd',
 'short_liquidations',
 'short_liquidations_usd',
 'open_interest',
 'buy_ratio',
 'buy_sell_ratio',
 'buy_volume',
 'sell_ratio',
 'sell_volume',
 'transaction_count',
 'hashrate',
 'liquidation_diff',
 'liquidation_usd_diff',
 'volume_diff',
 'buy_sell_volume_ratio',
 'volume_diff_pct_change_additive',
 'slope_change']

# check for interval using PACF



In [15]:
def shift_feature(
    df: pd.DataFrame,
    conti_cols: List[str],
    intervals: List[int],
) -> List[pd.Series]:
    """
    연속형 변수의 shift feature 생성
    Args:
        df (pd.DataFrame)
        conti_cols (List[str]): continuous colnames
        intervals (List[int]): shifted intervals
    Return:
        List[pd.Series]
    """
    df_shift_dict = [
        df[conti_col].shift(interval).rename(f"{conti_col}_{interval}")
        for conti_col in conti_cols
        for interval in intervals
    ]
    return df_shift_dict

# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[ _ for _ in range(1,5)]
)
# concat 하여 df 에 할당
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

In [ ]:
from typing import List
import pandas as pd

def rolling_feature(
   df: pd.DataFrame,
   conti_cols: List[str],
   intervals: List[int],
   funcs: List[str],
   **params,
) -> pd.DataFrame:
   """
   Create rolling features
   Args:
       df (pd.DataFrame): Sorted dataframe
       conti_cols (List[str]): continuous colnames
       intervals (List[str]): rolling window widths
       funcs (List[str]): aggregation functions e.g. ["mean", "median", "max"]
       **params: more input for rolling
   Returns:
       pd.DataFrame
   """
   df_rolling_list = [
       df[conti_col]
       .rolling(interval, **params)
       .agg({f"{conti_col}": func})
       .rename({conti_col: f"{conti_col}_{func}_{interval}"}, axis=1)
       for conti_col in conti_cols
       for interval in intervals
       for func in funcs
   ]
   return pd.concat(df_rolling_list, axis = 1)


In [16]:

df['open_interest_diff']=df['open_interest_3']-df['open_interest_2']
df['transaction_rolling_mean_6h'] = df['transaction_count'].rolling(window=6).mean()
df['transaction_rolling_mean_12h'] = df['transaction_count'].rolling(window=12).mean()
df['transaction_rolling_mean_24h'] = df['transaction_count'].rolling(window=24).mean()
df['hashrate_mean_6h'] = df['hashrate'].rolling(window=6).mean() 
df['hashrate_mean_12h'] = df['hashrate'].rolling(window=12).mean() 
df['hashrate_mean_1d'] = df['hashrate'].rolling(window=24).mean() 
df['hashrate_mean_7d'] = df['hashrate'].rolling(window=168).mean()  # 168 hours = 7 days
df['transaction_rolling_mean_diff-12-6']=df['transaction_rolling_mean_12h']-df['transaction_rolling_mean_6h']
df['transaction_rolling_mean_diff-24-6']=df['transaction_rolling_mean_24h']-df['transaction_rolling_mean_6h']
df['hashrate_mean_diff-7d-1d']=df['hashrate_mean_7d']-df['hashrate_mean_1d']
df.drop(columns=['transaction_rolling_mean_6h','transaction_rolling_mean_12h','transaction_rolling_mean_24h',
                 'hashrate_mean_6h','hashrate_mean_12h','hashrate_mean_1d','hashrate_mean_7d','open_interest_3','open_interest_2','open_interest_1'],inplace=True)

In [ ]:
#'block-interval_block_interval'
conti_cols = [
               'buy_volume',       
          'sell_volume'  ]

rolling_df = rolling_feature(
   df = df,
   conti_cols=conti_cols,
   intervals=[6,12,18,24,168],
   funcs=["mean", 'std','min','max'],
   min_periods = 1,
   closed = "left",
)

df=pd.concat([df, rolling_df], axis = 1)

In [ ]:
conti_cols = ['buy_volume', 'sell_volume']
periods = [6, 12, 18, 24, 168]  # Define the periods for pct_change

# Loop through each continuous column and each period
for col in conti_cols:
    for period in periods:
        # Create new columns for each percentage change with respective period
        df[f'{col}_pct_change_{period}'] = df[col].pct_change(period) * 100

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11552 entries, 0 to 11551
Columns: 115 entries, ID to hashrate_mean_diff-7d-1d
dtypes: datetime64[ns](1), float64(108), int64(5), object(1)
memory usage: 10.1+ MB


In [18]:
_target = df["target"]
df = df.ffill().fillna(-999).assign(target = _target)
# _type에 따라 train, test 분리
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8760 entries, 0 to 8759
Columns: 114 entries, ID to hashrate_mean_diff-7d-1d
dtypes: datetime64[ns](1), float64(108), int64(5)
memory usage: 7.7 MB


In [ ]:
# Step 1: Apply Window Slicing to Data Where Target is 0 or 3
df_filtered = train_df[(train_df['target'] == 0) | (train_df['target'] == 3)]
df_filtered
# Example window slicing function
# def window_slicing(time_series, window_size,step_size):
#     slices = []
#     for i in range(0,len(time_series) - window_size + 1,step_size):
#         window = time_series[i: i + window_size]
#         slices.append(window)
#     return np.array(slices)
def window_slicing(df: pd.DataFrame, window_size: int, step_size: int) -> pd.DataFrame:
    sliced_data = []
    
    for start in range(0, len(df) - window_size + 1, step_size):
        end = start + window_size
        window_df = df.iloc[start:end].copy()
        # window_df['window_start_index'] = start
        sliced_data.append(window_df)
    
    sliced_data_df = pd.concat(sliced_data, axis=0).reset_index(drop=True)
    return sliced_data_df

# Example window size (24 hours for 1 day window)
window_size = 24
step_size=12
sliced_data = window_slicing(df_filtered, window_size,step_size)
augmented_data=sliced_data
# augmented_data = pd.DataFrame()
# for i in sliced_data:
#     i_df = pd.DataFrame(i, columns=df_filtered.columns)
#     augmented_data = pd.concat([augmented_data, i_df], axis=0)
# augmented_data = augmented_data.astype(df_filtered.dtypes.to_dict())
# augmented_data.shape


In [ ]:
#Step 2: Apply Noise Injection to the Sliced Data (excluding 'id' and 'target')
def add_noise_to_data(df, noise_level=0.01):
    df_noisy = df.copy()

    # Exclude 'id' and 'target' columns from noise injectbion
    feature_columns = df.columns.difference(['ID', 'target','_type'])
    
    # Generate noise for the feature columns
    noise = np.random.normal(0, noise_level, size=df[feature_columns].shape)
    
    # Apply noise to feature columns
    df_noisy[feature_columns] = df[feature_columns] + noise
    
    return df_noisy
def noise_injection(df: pd.DataFrame, noise_level: float = 0.01, decay: float = 0.99, seed : int = 42) -> pd.DataFrame:
    df_noisy = df.copy()
    np.random.seed(seed)
    time_steps = np.arange(len(df_noisy))
    feature_columns = list(df.columns.difference(['ID', 'target','_type']+["liquidation_diffg", "liquidation_usd_diffg", "volume_diffg",'cross_volume_diff','cross_shifted_volume_diff',
                            'cross_signal_volume_diff',"Weekday","Weekend","Hour","Hour_1",
                           "Weekend_Hour"]) )
    for col in feature_columns:
        noise = np.random.normal(0, noise_level*df_noisy[col].std(), size=df_noisy[col].shape)
        decays = decay ** time_steps
        df_noisy[col] = df_noisy[col] + noise * decays
    
    return df_noisy

# Apply noise to the sliced data
augmented_noisy_data = noise_injection(augmented_data, noise_level=0.01,decay=0.99,seed=42)

In [ ]:
augmented_noisy_data.columns
train_1_2=train_df[(train_df['target'] != 0) & (train_df['target'] != 3)]
train_df_augmented=pd.concat([train_1_2,augmented_noisy_data])

In [ ]:
train_df_augmented.shape

In [ ]:

train_df_augmented['target'].value_counts()/len(train_df_augmented)

In [ ]:
#타겟 변수를 제외한 변수를 forwardfill, -999로 결측치 대체
# _target = df["target"]
# df = df.ffill().fillna(-999).assign(target = _target)

# # _type에 따라 train, test 분리
# train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
# test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

In [20]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis=1), 
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
    stratify=train_df["target"].astype(int)
)
x_train.shape

(7008, 112)

In [23]:
def train_lgmClassifier(x_train,x_valid,y_train,y_valid):
    

# lgb dataset
    train_data = lgb.Dataset(x_train, label=y_train)
    valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

# lgb params
    params = {
        "boosting_type": "gbdt",
        "objective": "multiclass",
        "metric": "multi_logloss",
        "num_class": 4,
        "num_leaves": 50,
        "learning_rate": 0.05,
        "n_estimators": 100,
        "random_state": 42,
        "verbose": 0,
    }

# lgb train
    lgb_model = lgb.train(
        params=params,
        train_set=train_data,
        valid_sets=valid_data,
    )

# lgb predict
    y_valid_pred = lgb_model.predict(x_valid)
    y_valid_pred_class = np.argmax(y_valid_pred, axis = 1)

# score check
    accuracy = accuracy_score(y_valid, y_valid_pred_class)
    auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")

    print(f"acc: {accuracy}, auroc: {auroc}")
    return lgb_model,accuracy,params
lgb_model,accuracy,params=train_lgmClassifier(x_train,x_valid,y_train,y_valid)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4463470319634703, auroc: 0.6119701684914071


In [ ]:
    # Confusion Matrix
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
y_valid_pred = lgb_model.predict(x_valid)
y_valid_pred_class = np.argmax(y_valid_pred, axis = 1)
conf_matrix = confusion_matrix(y_valid, y_valid_pred_class)

 # Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix")
plt.ylabel("Actual Class")
plt.xlabel("Predicted Class")
plt.show()



In [ ]:
def objective(trial):

    # Suggest hyperparameters
    params = {
        "boosting_type": "gbdt",
        "objective": "multiclass",
        "metric": "multi_logloss",
        "num_class": 4,  # Multiclass problem with 4 classes
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "random_state": 42,
        "verbose": -1,
    }
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    
    for train_idx, valid_idx in skf.split(x_train, y_train):
        # Use .iloc for pandas DataFrame indexing
        x_train_fold, x_valid_fold = x_train.iloc[train_idx], x_train.iloc[valid_idx]
        y_train_fold, y_valid_fold = y_train.iloc[train_idx], y_train.iloc[valid_idx]
        
        # Create the LightGBM dataset
        train_data = lgb.Dataset(x_train_fold, label=y_train_fold)
        valid_data = lgb.Dataset(x_valid_fold, label=y_valid_fold, reference=train_data)
        
        # Train the model with the specified parameters
        lgb_model = lgb.train(
            params=params,                # Pass the hyperparameters
            train_set=train_data,         # Training set
            valid_sets=[valid_data],      # Validation set
        )
        
        # Predict probabilities for the validation set
        y_pred_prob = lgb_model.predict(x_valid_fold, num_iteration=lgb_model.best_iteration)
        
        # Convert probabilities to class predictions
        y_pred_valid = np.argmax(y_pred_prob, axis=1)
        
        # Calculate accuracy
        accuracy = accuracy_score(y_valid_fold, y_pred_valid)
        accuracies.append(accuracy)
    
    # Return the mean accuracy across all folds
    return np.mean(accuracies)

# Initialize and run the study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Output the results of the best trial
trial = study.best_trial
print(f"Sampler is {study.sampler.__class__.__name__}")
print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))



In [24]:
from sklearn.ensemble import RandomForestClassifier
def train_RandomForestClassifier(x_train, x_valid, y_train, y_valid):    
    """Train and validate a RandomForestClassifier model.

    Args:        
        x_train (pd.DataFrame): Training feature data.    
        x_valid (pd.DataFrame): Validation feature data.    
        y_train (pd.Series): Training target labels.    
        y_valid (pd.Series): Validation target labels    
    """    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(x_train, y_train) 

    pred_proba =  model.predict_proba(x_train)
    print("train accuracy:{:5.10f}".format(accuracy_score(y_train, np.argmax(pred_proba, axis = 1))))
    print("train auroc :{:5.10f}".format(roc_auc_score(y_train, pred_proba, multi_class="ovr")))

    pred_proba =  model.predict_proba(x_valid)
    print("valid accuracy :{:5.10f}".format(accuracy_score(y_valid, np.argmax(pred_proba, axis = 1))))
    print("valid auroc :{:5.10f}".format(roc_auc_score(y_valid, pred_proba, multi_class="ovr")))
train_RandomForestClassifier(x_train, x_valid, y_train, y_valid)

train accuracy:1.0000000000
train auroc :1.0000000000
valid accuracy :0.4389269406
valid auroc :0.5868145443


In [25]:
importance = lgb_model.feature_importance()
feature_names=train_df.drop(["target", "ID"], axis = 1).columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
})

# Sort by importance (optional)
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the DataFrame
print(feature_importance_df[:30])
# a=feature_importance_df[:100]
# train_df[a.loc[:,'Feature']]

                                Feature  Importance
100   volume_diff_pct_change_additive_1         371
110  transaction_rolling_mean_diff-24-6         360
108                  open_interest_diff         350
103   volume_diff_pct_change_additive_4         350
7                         open_interest         346
101   volume_diff_pct_change_additive_2         332
111            hashrate_mean_diff-7d-1d         331
102   volume_diff_pct_change_additive_3         331
10                           buy_volume         329
82                           hashrate_3         328
107                      slope_change_4         318
22           volume_diff_pct_change_12h         317
12                          sell_volume         317
55                      open_interest_4         308
83                           hashrate_4         304
28       liquidation_diff_pct_change_3h         300
23            volume_diff_pct_change_6h         300
105                      slope_change_2         298
109  transac

In [ ]:
print(feature_importance_df[:30])

In [ ]:
from sklearn.model_selection import StratifiedKFold  # For stratified cross-validation
from sklearn.feature_selection import SelectKBest, f_classif  # For feature selection

pred_lgm = []
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation with stratification
for train_idx, val_idx in k_fold.split(train_df_augmented.drop(['ID', "target"], axis=1), train_df_augmented["target"]):
    # Train and validation sets
    x_tr = train_df_augmented.drop(["ID", "target"], axis=1).iloc[train_idx]  # Dropping both 'ID' and 'target'
    y_tr = train_df_augmented['target'].iloc[train_idx]
    x_val = train_df_augmented.drop(["ID", "target"], axis=1).iloc[val_idx]  # Dropping both 'ID' and 'target'
    y_val = train_df_augmented['target'].iloc[val_idx]

    # Select top 200 features using SelectKBest with f_classif
    # n_feature = 200
    # selector1 = SelectKBest(f_classif, k=n_feature)
    # x_tr_sel = selector1.fit_transform(x_tr, y_tr)
    # x_val_sel = selector1.transform(x_val)

    # Assuming train_lgmClassifier is a function you defined to train LightGBM and return accuracy
    train_RandomForestClassifier(x_tr, x_val, y_tr, y_val)

    # Store the accuracy for each fold


In [ ]:
params

# No feature selection train

In [ ]:
x_train = train_df.drop(["target", "ID"], axis = 1)
y_train = train_df["target"].astype(int)
train_data = lgb.Dataset(x_train, label=y_train)
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
)

In [ ]:
x_train = train_df.drop(["target", "ID"], axis = 1)
y_train = train_df["target"].astype(int)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train) 

# Feature selection train

In [ ]:

x_train = train_df_augmented.drop(["target", "ID"], axis = 1)
y_train = train_df_augmented["target"].astype(int)
n_feature = 200
selector1 = SelectKBest(f_classif, k=n_feature)
x_tr_sel = selector1.fit_transform(x_train, y_train)
train_data = lgb.Dataset(x_tr_sel, label=y_train)
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
)

# No feature selection

In [ ]:

y_test_pred = lgb_model.predict(test_df.drop(["target", "ID"], axis = 1))
y_test_pred_class = np.argmax(y_test_pred, axis = 1)

In [ ]:
y_test_pred = model.predict(test_df.drop(["target", "ID"], axis=1))
y_test_pred_proba = model.predict_proba(test_df.drop(["target", "ID"], axis=1))
y_test_pred_class = np.argmax(y_test_pred_proba, axis=1)

# Feature anova top 200 selection

In [ ]:
# selected_features = x_train.columns[selector1.get_support()]
x_test=test_df.drop(["target", "ID"], axis = 1)
x_test_sel = selector1.transform(x_test)

y_test_pred = lgb_model.predict(x_test_sel)
y_test_pred_class = np.argmax(y_test_pred, axis = 1)

In [ ]:
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df.to_csv("output5.csv", index=False)